In [29]:
import pandas as pd
import numpy as np

In [ ]:
vendas = pd.read_csv("C:/Users/augus/OneDrive/desktop/estudos/faculdade/fase5/fiap_fase5_trabalho/data/vendas_linha_petshop_2020_2024.csv",sep=";", decimal=",", encoding="latin1")

In [ ]:
def ajustar_tipos(df: pd.DataFrame) -> pd.DataFrame:
    """
    Realiza cast de colunas do DataFrame para os tipos adequados,
    tratando valores monetários, nulos, strings vazias e espaços.
    """
    tipos = {
        "cod_pedido": "Int64",  # inteiro com suporte a nulos
        "regiao_pais": "string",
        "produto": "string",
        "valor": "float64",
        "quantidade": "Int64",  # inteiro com suporte a nulos
        "valor_total_bruto": "float64",
        "data": "datetime64[ns]",
        "estado": "string",
        "formapagto": "string",
        "centro_distribuicao": "string",
        "responsavelpedido": "string",
        "valor_comissao": "float64",
        "lucro_liquido": "float64",
        "categoriaprod": "string",
    }

    for col, tipo in tipos.items():
        if col not in df.columns:
            continue

        if "float" in tipo:
            # Converte para string, limpa, substitui vazios/traços por NaN
            df[col] = (
                df[col]
                .astype(str)
                .str.strip()
                .replace({"": np.nan, " ": np.nan, "-": np.nan, "nan": np.nan, "None": np.nan})
                .str.replace(r"[^\d.,-]", "", regex=True)  # remove símbolos tipo R$
                .str.replace(",", ".", regex=False)       # vírgula -> ponto
            )
            # se ainda restar string vazia, vira NaN
            df[col] = pd.to_numeric(df[col], errors="coerce")

        elif tipo == "Int64":
            df[col] = (
                df[col]
                .astype(str)
                .str.strip()
                .replace({"": pd.NA, " ": pd.NA, "-": pd.NA, "nan": pd.NA, "None": pd.NA})
            )
            df[col] = pd.to_numeric(df[col], errors="coerce").astype("Int64")

        elif tipo == "datetime64[ns]":
            df[col] = pd.to_datetime(df[col], errors="coerce", dayfirst=True)

        else:
            df[col] = df[col].replace({"": pd.NA, " ": pd.NA}).astype(tipo)

    return df


In [ ]:
df_ajustado.to_parquet(
    "C:/Users/augus/OneDrive/desktop/estudos/faculdade/fase5/fiap_fase5_trabalho/data/vendas_linha_petshop_2020_2024.parquet",
    engine="fastparquet"
)

In [27]:
publi = pd.read_csv("C:/Users/augus/OneDrive/desktop/estudos/faculdade/fase5/fiap_fase5_trabalho/data/Gastos_Publicidade_MelhoresCompras.csv",sep=",", decimal=".", encoding="latin1")

In [30]:
def ajustar_tipos_marketing(df: pd.DataFrame) -> pd.DataFrame:
    """
    Ajusta os tipos de dados do dataset de marketing.
    """
    # Converte campos simples
    df["Ano"] = pd.to_numeric(df["Ano"], errors="coerce").astype("Int64")
    df["Mes"] = pd.to_numeric(df["Mes"], errors="coerce").astype("Int64")
    df["Tipo de Midia"] = df["Tipo de Midia"].astype("string")
    df["Previsao Inicial de Aumento de Vendas (em mil unidades)"] = (
        pd.to_numeric(df["Previsao Inicial de Aumento de Vendas (em mil unidades)"], errors="coerce")
        .astype("Int64")
    )

    # Tratamento especial para valores monetários
    df["Gastos com Publicidade (em R$)"] = (
        df["Gastos com Publicidade (em R$)"]
        .astype(str)
        .str.strip()
        .replace({"": np.nan, "nan": np.nan, "-": np.nan, "None": np.nan})
        .str.replace(r"[^\d,.-]", "", regex=True)   # remove R$, espaços etc.
        .str.replace(",", ".", regex=False)         # vírgula -> ponto decimal
    )
    df["Gastos com Publicidade (em R$)"] = pd.to_numeric(
        df["Gastos com Publicidade (em R$)"], errors="coerce"
    ).astype("float64")

    return df


In [31]:
publi_ajustada = ajustar_tipos_marketing(publi)

In [32]:
publi_ajustada.to_parquet("C:/Users/augus/OneDrive/desktop/estudos/faculdade/fase5/fiap_fase5_trabalho/data/Gastos_Publicidade_MelhoresCompras.parquet",engine="fastparquet")